In [17]:
# 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import asyncio
import json
import nest_asyncio
import pandas as pd
import re
from tqdm import tqdm
nest_asyncio.apply()

In [11]:
urls = pd.read_csv("../../data/K_naver_monthlyTop4000.csv")

In [12]:
urls

,Unnamed: 0,0
0,0,https://n.news.naver.com/mnews/article/003/001...
1,1,https://n.news.naver.com/mnews/article/055/000...
2,2,https://n.news.naver.com/mnews/article/016/000...
3,3,https://n.news.naver.com/mnews/article/018/000...
4,4,https://n.news.naver.com/mnews/article/421/000...
...,...,...
24674,24674,https://n.news.naver.com/mnews/article/018/000...
24675,24675,https://n.news.naver.com/mnews/article/417/000...
24676,24676,https://n.news.naver.com/mnews/article/018/000...
24677,24677,https://n.news.naver.com/mnews/article/417/000...


In [13]:
test = urls.iloc[:, 1:3]
test

,0
0,https://n.news.naver.com/mnews/article/003/001...
1,https://n.news.naver.com/mnews/article/055/000...
2,https://n.news.naver.com/mnews/article/016/000...
3,https://n.news.naver.com/mnews/article/018/000...
4,https://n.news.naver.com/mnews/article/421/000...
...,...
24674,https://n.news.naver.com/mnews/article/018/000...
24675,https://n.news.naver.com/mnews/article/417/000...
24676,https://n.news.naver.com/mnews/article/018/000...
24677,https://n.news.naver.com/mnews/article/417/000...


In [14]:
# 함수 정의
def get_comments(refer_url, comment_url) : # 댓글 목록을 json 형태로 받아오는 함수
    comments = []
    next = None
    # 처음엔 댓글 개수를 모르므로 충분히 큰 수를 넣어 줌
    comment_count = 10e6
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15',
        'referer': refer_url
    }
    
    # 수집한 댓글 수가 첫번째에 수집한 총 댓글 수 보다 많다면 반복을 종료합니다.
    while len(comments) < comment_count :
        comment_url_next = comment_url + '&moreParam.next=' + next if next else comment_url
        res = requests.get(comment_url_next, headers=headers)
        dic = json.loads(res.text[res.text.index('(')+1:-2])
        comments.extend(list(map(lambda x : {
            'url' : refer_url,
            'text': x['contents'],
            'reply_count' : x['replyCount'], 
            'uid': x['idNo'],
            'uname' : x['userName'],
            'like': x['sympathyCount'], 
            'dislike': x['antipathyCount'],
            'c_time' : x['modTime'],
            'cid': x['commentNo'],
            'pid' : x['parentCommentNo'] 
            }, dic['result']['commentList'])))
        comment_count = dic['result']['count']['comment']
        next = dic['result']['morePage']['next'] if comment_count else None
    # 필터로 삭제된 댓글을 걸러줍니다
    comments=list(filter(lambda x: len(x['text']), comments))
    return comments



In [15]:
def get_data(oid, aid) :
    try :
        # refer_url: 댓글 보기를 누르면 나오는 댓글 페이지 주소
        # comment_url: 네트워크 탭에서 확인 가능한 동적으로 생성되는 주소
        refer_url = f'https://n.news.naver.com/mnews/article/comment/{oid}/{aid}?sid=100'
        comment_url = f'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m3&pool=cbox5&_cv=20220623185938&_callback=jQuery33103200637113167857_1656171100524&lang=ko&country=KR&objectId=news{oid}%2C{aid}&categoryId=&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&initialize=true&userType=&useAltSort=true&replyPageSize=20&followSize=100&sort=new&includeAllStatus=true&_=1656171100525'

        comments = get_comments(refer_url, comment_url)
        comments = pd.DataFrame(comments)
        # 댓글 수가 0개인 기사를 어떻게 할지 추후에 결정해야 함
        return comments
    except :
        return None


In [16]:
df = pd.DataFrame()
for url in tqdm(test['0']):
    try:
        oid = re.findall('article/([^\*]*)/', str(url))[0]
        aid = re.findall(f'{oid}/([^\*]*)[?]', str(url))[0]
        sid = url[-3:]
        temp = get_data(oid, aid)
        temp['sid'] = sid
        df =pd.concat([df, temp])
    except:
        print(url)

  0%|          | 61/24679 [00:23<1:33:27,  4.39it/s]

https://entertain.naver.com/read?oid=018&aid=0005136161


  0%|          | 69/24679 [00:25<1:28:19,  4.64it/s]

https://entertain.naver.com/read?oid=009&aid=0004914053


  1%|          | 152/24679 [00:43<1:18:44,  5.19it/s]

...

100%|█████████▉| 24556/24679 [1:37:01<00:15,  8.01it/s]

https://sports.news.naver.com/news.nhn?oid=442&aid=0000157087


100%|█████████▉| 24648/24679 [1:37:23<00:06,  4.97it/s]

https://sports.news.naver.com/news.nhn?oid=442&aid=0000156970


100%|██████████| 24679/24679 [1:37:30<00:00,  4.22it/s]


In [18]:
df.reset_index(drop = True, inplace = True)
df

,url,text,reply_count,uid,uname,like,dislike,c_time,cid,pid,sid
0,https://n.news.naver.com/mnews/article/comment...,대한민국의 차량들이 너무 많다 정말 1가구당 차량수 1대로 제한해야한다 \n이대론 ...,0.0,2aqSG,jung****,0.0,2.0,2022-02-01T09:45:57+0900,748388986631749886,748388986631749886,101
1,https://n.news.naver.com/mnews/article/comment...,지들 잇속만 챙길라고 자사 주식 팔아치우는 카카오 경영진 새끼들은 입 다물어라......,0.0,xScD,mizn****,1.0,0.0,2022-01-31T18:35:36+0900,748330336135413976,748330336135413976,101
2,https://n.news.naver.com/mnews/article/comment...,카카오는 상도없는 장사꾼 이다 사회적 특혜를 준 현 정부는 그 댓가를 받고 국민의 ...,0.0,5KpGf,runj****,2.0,0.0,2022-01-31T15:28:38+0900,748318291084181771,748318291084181771,101
3,https://n.news.naver.com/mnews/article/comment...,페이 ceo가 싹 처분하고 날랐다던 카카오?,0.0,1yf7P,city****,0.0,0.0,2022-01-31T13:47:59+0900,748311807143182710,748311807143182710,101
4,https://n.news.naver.com/mnews/article/comment...,먹튀 지대로 하겟네,0.0,64081,kayk****,0.0,0.0,2022-01-31T12:54:30+0900,748308362663297206,748308362663297206,101
...,...,...,...,...,...,...,...,...,...,...,...
212917,https://n.news.naver.com/mnews/article/comment...,말로만 떠들어대네 ㅋㅋ 진짜 말이야 방구야,0.0,1jgyE,sho2****,0.0,0.0,2022-12-20T09:10:00+0900,778259026487017477,778259026487017477,105
212918,https://n.news.naver.com/mnews/article/comment...,ㅋㅋㅋㅋㅋㅋㅋ 뭔소리나,0.0,2LH5k,semo****,0.0,0.0,2022-12-20T07:08:37+0900,778251207113179288,778251207113179288,105
212919,https://n.news.naver.com/mnews/article/comment...,후회와 책망하지말고 명확한 해결책을 내고 한두번 있었던 일 아닌거 다 아니깐 기사로...,0.0,7uHcQ,kky0****,0.0,0.0,2022-12-20T09:29:28+0900,778260281355665639,778260281355665639,105
212920,https://n.news.naver.com/mnews/article/comment...,말로만 떠들어대네 ㅋㅋ 진짜 말이야 방구야,0.0,1jgyE,sho2****,0.0,0.0,2022-12-20T09:10:00+0900,778259026487017477,778259026487017477,105


In [21]:
df.to_pickle('../../data/K_naver_comments_monthlyTop4000.pkl')
df.to_csv('../../data/K_naver_comments_monthlyTop4000.csv')